# 해당 파일 설명
이 코드를 실행하기 위한 파일, /convert 경로에 다음의 파일이 필요합니다.

1. FC_data

이 코드를 모두 실행하고 나면, /convert 경로에 다음의 두 파일이 생성됩니다.

1. ALL_FC_DATA_MEAN.csv

## 추가 설명
10개의 관측소의 예보 데이터 파일을

하나의 ALL_FC_DATA_MEAN 로 정리합니다.



# 1. Google Drive Mount

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# 2. Define Directory

In [ ]:
# 사용하기 전 변경해주어야 하는 파라미터
BASE_WORKING_DIR = "/content/gdrive/My Drive/weather_bigcontest/4_code/"
USERNAME = "park"

In [ ]:
import os
# 사용하는 디렉토리 정의
DATA_BASE_PATH = os.path.join(BASE_WORKING_DIR, "data")
DATA_RAW_PATH = os.path.join(DATA_BASE_PATH, "raw")
DATA_EXPORT_PATH = os.path.join(DATA_BASE_PATH, "export")
DATA_CONV_PATH = os.path.join(DATA_BASE_PATH, 'converted')
WORKING_DIR = os.path.join(BASE_WORKING_DIR, USERNAME)

print('Current Work Directory :',  os.getcwd(), '\n')
print('Username :',  USERNAME, '\n')

Current Work Directory : /content/gdrive/.shortcut-targets-by-id/1xoPHwByu_twRzIwOuduO65YG5PSh7kTJ/weather_bigcontest/4_code/park 

Username : park 



In [ ]:
# 데이터 무엇이 있는지 확인하자.
# 아래 코드를 수행하게 되면, "../input/"에서 리눅스 명령어 "ls"를 수행한 결과를 보여주게 된다. 이 때 나온 binary code를 "utf8"로 디코딩해서 보여준다.
from subprocess import check_output
print("===== Raw Data List =====\n", check_output(["ls", DATA_RAW_PATH]).decode("utf8"))
print("===== Export Data List =====\n" ,check_output(["ls", DATA_EXPORT_PATH]).decode("utf8"))
print("===== Converted Data List =====\n" ,check_output(["ls", DATA_CONV_PATH]).decode("utf8"))

===== Raw Data List =====
 plant1_train.csv
plant2_train.csv
plant2_train.gsheet
plant_test.csv
submit.csv
submit.gsheet

===== Export Data List =====
 dangjin_aws
forecast_data
seosan_asos
sinpyeong_aws
songak_forecast
songak_local
songsan_forecast
songsan_local

===== Converted Data List =====
 ALL_FC_DATA_MEAN.csv
auc_95_temp_save
backup
dangjin_weather.csv
data_all_time_out.csv
data_all_time_out_in_add_mea_ddhr.csv
data_all_time_out_in.csv
data_all_time_predict.csv
data_plant_train_trend_point.csv
date_trend_point.csv
FC_data
old_local
plant1_by_loc_with_datetime.csv
plant2_by_loc_with_datetime.csv
plant_test_all_weather_feature_add_cluster_add_out.csv
plant_test_all_weather_feature_add_cluster_add_out_in.csv
plant_test_all_weather_feature_add_cluster.csv
plant_test_all_weather_feature.csv
plant_test_with_datetime.csv
plant_train_all_weather_feature_add_cluster.csv
plant_train_all_weather_feature.csv
seosan_weather.csv
sinpyeong_weather.csv
songak_weather.csv
songak_weather_utc.csv

# 4. Import Library

In [ ]:
# 기본 데이터 라이브러리 로드
import sys # 시스템 파라미터에 접근할 수 있게 도와준다.
print("Python version: {}". format(sys.version))

import pandas as pd # 데이터 정제에 도움을 주는 라이브러리
print("pandas version: {}". format(pd.__version__))

import matplotlib # 매트랩에서 사용하는 시각화 도구를 사용할 수 있게 도와주는 시각화 도구
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np # 행렬 계산을 위해 필요한 라이브러리
print("NumPy version: {}". format(np.__version__))

import scipy as sp # 수학 관련 함수가 내장된 라이브러리
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display # 주피터 노트북에서 예쁘게 시각화 해주는 도구
print("IPython version: {}". format(IPython.__version__)) 

import sklearn # 각종 통계 도구와 머신 러닝 알고리즘이 내장되어 있는 라이브러리
print("scikit-learn version: {}". format(sklearn.__version__))

# 파이썬 내장 라이브러리
import random
import time
import os


# Jupyter Notebook 이나 ipython 을 사용하다보면 향후 버전이 올라갈 때 변경될 사항 등을 알려주는 경고 메시지(warning message)를 뜨지 않게 해준다.
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

Python version: 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]
pandas version: 1.0.5
matplotlib version: 3.2.2
NumPy version: 1.18.5
SciPy version: 1.4.1
IPython version: 5.5.0
scikit-learn version: 0.22.2.post1
-------------------------


In [ ]:
# 모델링 라이브러리, 수학 계산 툴 로드
# 일반적인 모델링 라이브러리
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
import lightgbm

# 모델링 시 헬퍼 함수들
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

# 시각화 도구
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

# 시각화 도구 default 세팅
# 주피터 노트북에서 plot 결과를 볼 수 있게 해준다.
%matplotlib inline
mpl.style.use('ggplot') # matplotlib에서 plot되는 결과를 선택할 수 있다.
sns.set_style('white') # seaborn에서 사용할 style을 설정할 수 있다.
pylab.rcParams['figure.figsize'] = 12,8 # plot의 크기와 선 등의 기본 값을 설정할 수 있다.

# 시각화 Style 설정하기
# print(plt.style.available)
# ['bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'seaborn', 'Solarize_Light2', 'tableau-colorblind10', '_classic_test']

plt.style.use(['fivethirtyeight'])
sns.set_style('darkgrid')

# 5. Read Raw Data

In [ ]:
path_dir = os.path.join(DATA_CONV_PATH, "FC_data")
file_list = os.listdir(path_dir)
FC_PATH = []
for file in file_list:
  if 'utc' in file:
    FC_PATH.append(os.path.join(path_dir, file))
FC_PATH

['/content/gdrive/My Drive/weather_bigcontest/4_code/data/converted/FC_data/songsan_weather_FC_utc.csv',
 '/content/gdrive/My Drive/weather_bigcontest/4_code/data/converted/FC_data/songak_weather_FC_utc.csv',
 '/content/gdrive/My Drive/weather_bigcontest/4_code/data/converted/FC_data/sinpyeong_weather_FC_utc.csv',
 '/content/gdrive/My Drive/weather_bigcontest/4_code/data/converted/FC_data/dangjin1_weather_FC_utc.csv',
 '/content/gdrive/My Drive/weather_bigcontest/4_code/data/converted/FC_data/dangjin2_weather_FC_utc.csv',
 '/content/gdrive/My Drive/weather_bigcontest/4_code/data/converted/FC_data/dangjin3_weather_FC_utc.csv',
 '/content/gdrive/My Drive/weather_bigcontest/4_code/data/converted/FC_data/godae_weather_FC_utc.csv',
 '/content/gdrive/My Drive/weather_bigcontest/4_code/data/converted/FC_data/emam_weather_FC_utc.csv',
 '/content/gdrive/My Drive/weather_bigcontest/4_code/data/converted/FC_data/susuk_weather_FC_utc.csv',
 '/content/gdrive/My Drive/weather_bigcontest/4_code/data/

In [ ]:
path = FC_PATH[0]
data = pd.read_csv(path, encoding = 'cp949')
data = data.groupby(['date', 'time']).mean()
idx = list(data.index)
for i in range(len(idx)):
  idx[i] = list(idx[i])
idx = np.array(idx)
data['date'] = idx[:, 0]
data['time'] = idx[:, 1]
data.reset_index(drop=True, inplace=True)
data.drop(columns=['mea_order'], inplace=True)
data

,ta,hm,ws,date,time
0,1.900000,43.000000,2.9,2016-03-01,900.0
1,0.200000,63.000000,3.9,2016-03-01,1080.0
2,-0.933333,78.666667,1.9,2016-03-01,1260.0
3,-1.050000,79.500000,1.0,2016-03-02,0.0
4,-1.300000,80.400000,1.9,2016-03-02,180.0
...,...,...,...,...,...
11951,14.000000,35.000000,3.0,2020-04-03,720.0
11952,16.000000,25.000000,4.5,2020-04-03,900.0
11953,12.000000,40.000000,4.1,2020-04-03,1080.0
11954,8.000000,70.000000,2.6,2020-04-03,1260.0


In [ ]:
data = pd.read_csv(FC_PATH[0], encoding = 'cp949')
data = data.groupby(['date', 'time']).mean()
for i in range(len(idx)):
  idx[i] = list(idx[i])
idx = np.array(idx)
data['date'] = idx[:, 0]
data['time'] = idx[:, 1]
data.reset_index(drop=True, inplace=True)

FC_DATA = data[['date', 'time']]
for path in FC_PATH:
  data = pd.read_csv(path, encoding = 'cp949')
  data = data.groupby(['date', 'time']).mean()
  idx = list(data.index)
  for i in range(len(idx)):
    idx[i] = list(idx[i])
  idx = np.array(idx)
  data['date'] = idx[:, 0]
  data['time'] = idx[:, 1]
  data.reset_index(drop=True, inplace=True)
  data.drop(columns=['mea_order'], inplace=True)
  name = path.split('/')[-1].split('_')[0]
  data.columns = [name+'_ta', name+'_hm', name+'_ws','date','time']
  FC_DATA = pd.merge(FC_DATA, data, how='left', on=['date', 'time'])
FC_DATA

,date,time,songsan_ta,songsan_hm,songsan_ws,songak_ta,songak_hm,songak_ws,sinpyeong_ta,sinpyeong_hm,sinpyeong_ws,dangjin1_ta,dangjin1_hm,dangjin1_ws,dangjin2_ta,dangjin2_hm,dangjin2_ws,dangjin3_ta,dangjin3_hm,dangjin3_ws,godae_ta,godae_hm,godae_ws,emam_ta,emam_hm,emam_ws,susuk_ta,susuk_hm,susuk_ws,suknam_ta,suknam_hm,suknam_ws
0,2016-03-01,900.0,1.900000,43.000000,2.9,2.000000,43.000000,2.8,1.900000,42.0,2.5,2.0,45.0,3.0,2.0,45.0,3.0,2.0,45.0,3.0,1.800000,46.0,3.0,2.0,45.0,3.0,2.200000,46.0,3.0,2.200000,46.0,3.0
1,2016-03-01,1080.0,0.200000,63.000000,3.9,0.100000,63.000000,3.8,0.500000,60.0,3.4,0.0,65.0,4.0,0.0,65.0,4.0,0.0,65.0,4.0,0.100000,65.0,4.0,0.0,65.0,4.0,0.300000,64.0,3.9,0.300000,64.0,3.9
2,2016-03-01,1260.0,-0.933333,78.666667,1.9,-1.133333,78.666667,1.8,-1.166667,77.0,1.5,-1.0,80.0,2.0,-1.0,80.0,2.0,-1.0,80.0,2.0,-0.766667,79.0,1.9,-1.0,80.0,2.0,-0.433333,79.0,2.0,-0.433333,79.0,2.0
3,2016-03-02,0.0,-1.050000,79.500000,1.0,-1.200000,80.000000,1.0,-1.225000,79.5,1.0,-1.0,80.0,1.0,-1.0,80.0,1.0,-1.0,80.0,1.0,-0.850000,78.5,1.0,-1.0,80.0,2.0,-0.450000,79.0,2.0,-0.450000,79.0,2.0
4,2016-03-02,180.0,-1.300000,80.400000,1.9,-1.580000,80.400000,1.9,-1.920000,79.2,1.6,-1.2,80.0,2.0,-1.2,80.0,2.0,-1.2,80.0,2.0,-0.940000,78.6,1.9,-1.2,80.0,2.0,-0.600000,78.0,2.0,-0.600000,78.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11951,2020-04-03,720.0,14.000000,35.000000,3.0,14.000000,35.000000,2.8,14.000000,35.0,2.8,14.0,35.0,2.3,14.0,35.0,2.3,14.0,35.0,2.3,14.000000,35.0,2.8,14.0,30.0,2.9,13.000000,40.0,2.9,13.000000,40.0,2.9
11952,2020-04-03,900.0,16.000000,25.000000,4.5,15.000000,25.000000,4.2,15.000000,25.0,4.3,15.0,30.0,3.1,15.0,30.0,3.1,15.0,30.0,3.1,15.000000,30.0,4.3,15.0,30.0,4.2,13.000000,35.0,4.2,13.000000,35.0,4.2
11953,2020-04-03,1080.0,12.000000,40.000000,4.1,12.000000,40.000000,3.7,12.000000,40.0,3.7,11.0,40.0,2.3,11.0,40.0,2.3,11.0,40.0,2.3,11.000000,45.0,3.9,11.0,45.0,3.4,10.000000,50.0,3.7,10.000000,50.0,3.7
11954,2020-04-03,1260.0,8.000000,70.000000,2.6,8.000000,65.000000,2.6,8.000000,65.0,2.3,8.0,70.0,1.5,8.0,70.0,1.5,8.0,70.0,1.5,8.000000,75.0,2.8,7.0,75.0,1.7,7.000000,80.0,2.8,7.000000,80.0,2.8


In [ ]:
FC_DATA.to_csv(os.path.join(DATA_CONV_PATH, 'ALL_FC_DATA_MEAN.csv'))

In [ ]:
FC_DATA['time'].unique()

array(['900.0', '1080.0', '1260.0', '0.0', '180.0', '360.0', '540.0',
       '720.0'], dtype=object)